# Test THREDDS using Magpie Authentication/Authorization against Twitcher proxy.

## Configuration parameters

In [1]:
# NBVAL_IGNORE_OUTPUT

from __future__ import print_function
import os
import uuid
import requests
print("Setup configuration parameters...")

PAVICS_HOST = os.getenv("PAVICS_HOST", "pavics.ouranos.ca").rstrip("/")
if not PAVICS_HOST:
    raise ValueError("Cannot run test without a PAVICS_HOST value.")

PAVICS_URL = "https://{}".format(PAVICS_HOST)
VERIFY_SSL = True if "DISABLE_VERIFY_SSL" not in os.environ else False
MAGPIE_URL = PAVICS_URL + "/magpie"
TWITCHER_PROXY = "/twitcher/ows/proxy"
TWITCHER_URL = PAVICS_URL + TWITCHER_PROXY
THREDDS_SERVICE = "thredds"
THREDDS_URL = "{}/{}".format(TWITCHER_URL, THREDDS_SERVICE)

HEADERS = {"Accept": "application/json", "Content-Type": "application/json"}

MAGPIE_ANONYMOUS_GROUP = "anonymous"
TEST_MAGPIE_ADMIN_USERNAME = os.getenv("TEST_MAGPIE_ADMIN_USERNAME")
TEST_MAGPIE_ADMIN_PASSWORD = os.getenv("TEST_MAGPIE_ADMIN_PASSWORD")
if not TEST_MAGPIE_ADMIN_USERNAME or not TEST_MAGPIE_ADMIN_PASSWORD:
    raise ValueError("Missing test admin credentials to run tests.")

print("  Will use Magpie URL:       [{}]".format(MAGPIE_URL))
print("  Will use Twitcher URL:     [{}]".format(TWITCHER_URL))
print("  Will use THREDDS service:  [{}]".format(THREDDS_SERVICE))


def response_msg(message, response, is_json=True):
    """Append useful response details to provided message."""
    if is_json:
        # magpie formatted error
        _body = response.json()
        _detail = _body.get("detail", _body.get("message", "<unknown>"))
    else:
        _body = response.text
        # twitcher formatted error
        if "<ExceptionText>" in _body:
            _detail = _body.split("<ExceptionText>")[-1].split("</ExceptionText>")[0].strip()
        # thredds formatted error
        elif "Error {" in _body:
            _detail = _body.split("message = ")[-1].split(";\n")[0].strip()
        else:
            _detail = "<stripped>"
    return "{} Response replied with ({}) [{}]".format(message, response.status_code, _detail)

# test URLs are reachable
resp = requests.get(MAGPIE_URL, verify=VERIFY_SSL, headers=HEADERS)
if not resp.status_code == 200:
    raise ValueError(response_msg("Could not reach MAGPIE_URL.", resp))
resp = requests.get(THREDDS_URL, verify=VERIFY_SSL)
if not resp.status_code == 200:
    raise ValueError(response_msg("Could not reach THREDDS_URL.", resp))

Setup configuration parameters...
  Will use Magpie URL:       [https://host-140-36.rdext.crim.ca/magpie]
  Will use Twitcher URL:     [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy]
  Will use THREDDS service:  [thredds]


## Validate test admin user can have needed access to adjust permissions and that anonymous group is valid

In [2]:
# NBVAL_IGNORE_OUTPUT

print("Validate that user has needed admin-level access... ", end="")

def login_user(username, password):
    _data = {"user_name": username, "password": password}
    _path = MAGPIE_URL + "/signin"
    _resp = requests.post(_path, json=_data, verify=VERIFY_SSL, headers=HEADERS)
    if _resp.status_code != 200:
        raise ValueError(response_msg("\nCould not login with provided test admin credentials.", _resp))
    return _resp.cookies

ADMIN_COOKIES = login_user(TEST_MAGPIE_ADMIN_USERNAME, TEST_MAGPIE_ADMIN_PASSWORD)

# test admin-level access with route that is admin only
path = MAGPIE_URL + "/users"
resp = requests.get(path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    raise ValueError(response_msg("\nProvided user credentials doesn't have administrative permissions.", resp))

path = "{}/groups/{}".format(MAGPIE_URL, MAGPIE_ANONYMOUS_GROUP)
resp = requests.get(path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    msg = "\nCould not validate Magpie anonymous group using name: [{}]".format(MAGPIE_ANONYMOUS_GROUP)
    raise ValueError(response_msg(msg, resp))

print("OK")

Validate that user has needed admin-level access... OK


## Define functions to clean test user and group after tests

Since we know that admin is correctly configured up to this point,
setup cleanup operations to remove test data after test execution.

In [3]:
# NBVAL_IGNORE_OUTPUT

TEST_GROUP_NAME = "test-auth-{!s}".format(uuid.uuid4())
TEST_USER_NAME = "test-user-{!s}".format(uuid.uuid4())
# password must not be displayed to keep minimal obfuscation
#   since test user name will be logged, and eventually receive slightly more access than 'anonymous',
#   don't give a chance for anyone to guess the credentials
TEST_USER_PWD = str(uuid.uuid4())
CLEANUP_CALLED = False

def cleanup_test_data(skip_fail=True):
    """Cleanup test elements created during the test.

    Only raise at the complete end to attempt removal of as much element as possible in case of failures.
    """
    global CLEANUP_CALLED
    CLEANUP_CALLED = True
    failed = False
    for item_type, item_name, item_value in [
        ("users", "TEST_USER_NAME", TEST_USER_NAME),
        ("groups", "TEST_GROUP_NAME", TEST_GROUP_NAME),
    ]:
        try:
            _path = "{}/{}/{}".format(MAGPIE_URL, item_type, item_value)
            _resp = requests.delete(_path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
            _head = "Cleanup of {} [{}]:".format(item_name, item_value)
            if _resp.status_code == 200:
                print(_head, "OK")
            elif _resp.status_code == 404:
                print(_head, "WARNING - does not exist (maybe it failed to be created in the first place?)")
                failed = True  # most definitely the test is already failing at this point anyway
            else:
                print(_head, "ERROR - could not be removed")
                failed = True
        except Exception as exc:
            print("Cleanup raised: [{}]".format(exc))
            failed = True
    if not skip_fail and failed:
        raise ValueError("Could not cleanup all test data.")


## Create a test group and test user that will not have admin access

In [4]:
# NBVAL_IGNORE_OUTPUT

print("Create test group and test user... ")
print("  Will use TEST_USER_NAME:  [{}]".format(TEST_USER_NAME))
print("  Will use TEST_GROUP_NAME: [{}]".format(TEST_GROUP_NAME))

data = {"group_name": TEST_GROUP_NAME}
path = MAGPIE_URL + "/groups"
resp = requests.post(path, json=data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 201:
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not create test group [{}]".format(TEST_GROUP_NAME), resp))

data = {"user_name": TEST_USER_NAME, "password": TEST_USER_PWD,
        "email": "{}@mail.com".format(TEST_USER_NAME), "group_name": TEST_GROUP_NAME}
path = MAGPIE_URL + "/users"
resp = requests.post(path, json=data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 201:
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not create test user [{}]".format(TEST_USER_NAME), resp))

print("Creation: OK")

Create test group and test user... 
  Will use TEST_USER_NAME:  [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Will use TEST_GROUP_NAME: [test-auth-1aa9f459-f38a-4478-85b1-551634239a65]
Creation: OK


## Create specific test resources under THREDDS service

Expected data structure as per:
    [https://github.com/bird-house/birdhouse-deploy/blob/master/birdhouse/scripts/bootstrap-testdata](
    https://github.com/bird-house/birdhouse-deploy/blob/master/birdhouse/scripts/bootstrap-testdata
    )

In [5]:
# NBVAL_IGNORE_OUTPUT

print("Create test resources under THREDDS service... ", end="")

path = MAGPIE_URL + "/services/{}".format(THREDDS_SERVICE)
resp = requests.get(path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not fetch THREDDS service details [{}].".format(THREDDS_SERVICE), resp))
thredds_res_id = resp.json()["service"]["resource_id"]

def create_or_get_resource(resource_name, resource_type, parent_id):
    _path = MAGPIE_URL + "/resources"
    _data = {"parent_id": parent_id, "resource_type": resource_type, "resource_name": resource_name}
    _resp = requests.post(_path, json=_data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
    _msg = "[name: {}, type: {}, parent: {}].".format(resource_name, resource_type, parent_id)
    if _resp.status_code == 409:
        _path = "{}/{}".format(_path, parent_id)
        _resp = requests.get(_path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
        if _resp.status_code != 200:
            cleanup_test_data()
            raise ValueError(response_msg("Could not retrieve resource expected to exist {}.".format(_msg), _resp))
        _resource = _resp.json()["resource"]
        _children = _resource["children"]
        for _, _child_info in _children.items():
            if _child_info["resource_name"] == resource_name:
                return _child_info
    elif _resp.status_code != 201:
        cleanup_test_data()
        raise ValueError(response_msg("\nCould not create resource {}.".format(_msg), _resp))
    else:
        return _resp.json()["resource"]
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not create or retrieve resource {}.".format(_msg), _resp))

birdhouse_res = create_or_get_resource("birdhouse", "directory", thredds_res_id)
testdata_res = create_or_get_resource("testdata", "directory", birdhouse_res["resource_id"])
secure_res = create_or_get_resource("secure", "directory", testdata_res["resource_id"])
taskmax_name = "tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc"
taskmax_res = create_or_get_resource(taskmax_name, "file", secure_res["resource_id"])

birdhouse_path = birdhouse_res["resource_name"]
birdhouse_catalog = "catalog/{}/catalog.html".format(birdhouse_path)
testdata_path = "{}/{}".format(birdhouse_path, testdata_res["resource_name"])
testdata_catalog = "catalog/{}/catalog.html".format(testdata_path)
secure_path = "{}/{}".format(testdata_path, secure_res["resource_name"])
secure_catalog = "catalog/{}/catalog.html".format(secure_path)
taskmax_path = "{}/{}".format(secure_path, taskmax_res["resource_name"])
taskmax_file = "fileServer/{}".format(taskmax_path)
taskmax_ncml = "ncml/{}".format(taskmax_path)
taskmax_dodsC = "dodsC/{}.html".format(taskmax_path)

print("OK")

Create test resources under THREDDS service... OK


## Create specific permissions for THREDDS resources to block public access to 'secure' directory

In [6]:
# NBVAL_IGNORE_OUTPUT

print("Create permissions on test resources... ", end="")

def set_permission(permission, resource, target_type, target_name):
    _res_id = resource["resource_id"]
    _path = "{}/{}/{}/resources/{}/permissions".format(MAGPIE_URL, target_type, target_name, _res_id)
    _data = {"permission": permission}
    _resp = requests.put(_path, json=_data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
    if _resp.status_code not in [200, 201]:
        _msg = "\nCleanup called before? {} (if called, following error could be expected)".format(CLEANUP_CALLED) + \
               "\nCould not set permission [{}] for [{}, {}] over resource [{}, {}]" \
               .format(permission, target_type, target_name, resource["resource_name"], _res_id)
        cleanup_test_data()
        raise ValueError(response_msg(_msg, _resp))


set_permission("browse-allow-recursive", birdhouse_res, "groups", MAGPIE_ANONYMOUS_GROUP)
set_permission("read-allow-recursive", birdhouse_res, "groups", MAGPIE_ANONYMOUS_GROUP)
set_permission("browse-deny-recursive", secure_res, "groups", MAGPIE_ANONYMOUS_GROUP)
set_permission("read-deny-recursive", secure_res, "groups", MAGPIE_ANONYMOUS_GROUP)

print("OK")

Create permissions on test resources... OK


## Verify allowed/blocked user access to secured resources

In [7]:
# NBVAL_IGNORE_OUTPUT

print("Testing allowed/blocked access to secured THREDDS resources...")

TEST_USER_COOKIES = login_user(TEST_USER_NAME, TEST_USER_PWD)
ANONYMOUS_COOKIES = {}


def has_access(resource_path, user_cookies, user_name):
    _path = "{}/{}".format(THREDDS_URL, resource_path)
    _resp = requests.get(_path, verify=VERIFY_SSL, cookies=user_cookies)
    if _resp.status_code in [200, 401, 403]:
        is_allowed = _resp.status_code == 200
        str_allowed = "Allowed" if is_allowed else "Denied"
        print("Detail:\n"
              "  Resource: [{}]\n"
              "  User:     [{}]\n"
              "  Access:   [{}]".format(_path, user_name, str_allowed))
        return is_allowed
    cleanup_test_data()
    _msg = "Unexpected status code during access to resource [{}]".format(_path)
    raise ValueError(response_msg(_msg, _resp, is_json=False))

# admin always has full access regardless of permissions
# avoid potential leak of critical admin credentials here by using a fill-in value
assert has_access(birdhouse_catalog, ADMIN_COOKIES, "<TEST_MAGPIE_ADMIN_USERNAME>")
assert has_access(testdata_catalog, ADMIN_COOKIES, "<TEST_MAGPIE_ADMIN_USERNAME>")
assert has_access(secure_catalog, ADMIN_COOKIES, "<TEST_MAGPIE_ADMIN_USERNAME>")
assert has_access(taskmax_file, ADMIN_COOKIES, "<TEST_MAGPIE_ADMIN_USERNAME>")
assert has_access(taskmax_ncml, ADMIN_COOKIES, "<TEST_MAGPIE_ADMIN_USERNAME>")
assert has_access(taskmax_dodsC, ADMIN_COOKIES, "<TEST_MAGPIE_ADMIN_USERNAME>")

# test user has different access depending on permissions applied
# for debugging and cleanup, print its generated name, but not the password
assert has_access(birdhouse_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(testdata_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(secure_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(taskmax_file, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(taskmax_ncml, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(taskmax_dodsC, TEST_USER_COOKIES, TEST_USER_NAME)

assert has_access(birdhouse_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert has_access(testdata_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(secure_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_file, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_ncml, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_dodsC, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")

print("Expected secured access to all resources: OK")

Testing allowed/blocked access to secured THREDDS resources...
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/catalog.html]
  User:     [<TEST_MAGPIE_ADMIN_USERNAME>]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/catalog.html]
  User:     [<TEST_MAGPIE_ADMIN_USERNAME>]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/secure/catalog.html]
  User:     [<TEST_MAGPIE_ADMIN_USERNAME>]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/fileServer/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
  User:     [<TEST_MAGPIE_ADMIN_USERNAME>]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/ncml/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_r

## Verify that directories become browsable after update of test user group access

In [8]:
# NBVAL_IGNORE_OUTPUT

# Anonymous user will still not have access to resources,
# but users that are member of the allowed test group will be allowed to browse (even if not admins).
# Read (fetch of files) remains blocked for now.
set_permission("browse-allow-recursive", secure_res, "groups", TEST_GROUP_NAME)

assert has_access(birdhouse_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(testdata_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(secure_catalog, TEST_USER_COOKIES, TEST_USER_NAME)     # becomes accessible
assert has_access(taskmax_ncml, TEST_USER_COOKIES, TEST_USER_NAME)       # becomes accessible (ncml = browse)
assert not has_access(taskmax_file, TEST_USER_COOKIES, TEST_USER_NAME)   # still blocked (fileServer = read)
assert not has_access(taskmax_dodsC, TEST_USER_COOKIES, TEST_USER_NAME)  # still blocked (dodsC = read)

# following remains the same as previous cell, but validate to make sure
assert has_access(birdhouse_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert has_access(testdata_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(secure_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_ncml, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_file, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_dodsC, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")

print("Expected browse access to all resources: OK")


Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/secure/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/ncml/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/fileServer/birdhouse/testdata/secure/tasmax_Amon_MPI

## Verify that directories become browsable after update of test user group access

In [9]:
# NBVAL_IGNORE_OUTPUT

# Additionally grant read access, test-user should now be allowed to fetch of files
set_permission("read-allow-recursive", secure_res, "groups", TEST_GROUP_NAME)

# Combined permissions resolution grant full access to users member of test group
assert has_access(birdhouse_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(testdata_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(secure_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_ncml, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_file, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_dodsC, TEST_USER_COOKIES, TEST_USER_NAME)

# following remains the same as previous cell, but validate to make sure
assert has_access(birdhouse_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert has_access(testdata_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(secure_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_ncml, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_file, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_dodsC, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")

print("Expected read access to all resources: OK")

Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/secure/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/ncml/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/fileServer/birdhouse/testdata/secure/tasmax_Amon_MPI

## Verify additional read access added specifically to test group

In [10]:
# NBVAL_IGNORE_OUTPUT

# Additionally grant read access, test-user should now be allowed to fetch of files
set_permission("read-allow-recursive", secure_res, "groups", TEST_GROUP_NAME)

# Combined permissions resolution grant full access to users member of test group
assert has_access(birdhouse_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(testdata_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(secure_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_ncml, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_file, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_dodsC, TEST_USER_COOKIES, TEST_USER_NAME)

# following remains the same as previous cell, but validate to make sure
assert has_access(birdhouse_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert has_access(testdata_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(secure_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_ncml, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_file, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_dodsC, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")

print("Expected read access to all resources: OK")


Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/secure/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/ncml/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/fileServer/birdhouse/testdata/secure/tasmax_Amon_MPI

In [11]:
# NBVAL_IGNORE_OUTPUT

# Additionally grant read access, test-user should now be allowed to fetch of files
set_permission("read-allow-recursive", secure_res, "groups", TEST_GROUP_NAME)

# Combined permissions resolution grant full access to users member of test group
assert has_access(birdhouse_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(testdata_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(secure_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_ncml, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_file, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_dodsC, TEST_USER_COOKIES, TEST_USER_NAME)

# following remains the same as previous cell, but validate to make sure
assert has_access(birdhouse_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert has_access(testdata_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(secure_catalog, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_ncml, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_file, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")
assert not has_access(taskmax_dodsC, ANONYMOUS_COOKIES, "<ANONYMOUS_USER>")

print("Expected read access to all resources: OK")

Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/catalog/birdhouse/testdata/secure/catalog.html]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/ncml/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
  User:     [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]
  Access:   [Allowed]
Detail:
  Resource: [https://host-140-36.rdext.crim.ca/twitcher/ows/proxy/thredds/fileServer/birdhouse/testdata/secure/tasmax_Amon_MPI

In [12]:
# NBVAL_IGNORE_OUTPUT

# final cleanup, everything is expected to have worked up to here,
# so force failure to ensure we return to original 'clean' state
cleanup_test_data(skip_fail=False)
print("All tests: OK")


Cleanup of TEST_USER_NAME [test-user-5d73303e-8a3a-48e9-94bb-197bcf51680e]: OK
Cleanup of TEST_GROUP_NAME [test-auth-1aa9f459-f38a-4478-85b1-551634239a65]: OK
All tests: OK
